### Linear Model Based on Lasso Regression

#### Lasso has told us which features to keep, so let's create a model with that guidance and see how it does!

In [144]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate

In [10]:
#read in our dataframe from our Regression: Main notebook
linear_df = pd.read_pickle('linear_df.pickle')

In [11]:
linear_df.shape

(847, 18)

In [9]:
columns = (['domestic_opening_gross'],
['foreign_market_count'],
['Adventure'],
['Comedy'],
['domestic_opening_gross']**2
['domestic_opening_gross']*['foreign_market_count'],
['domestic_opening_gross']*['Years_Since_Release'],
['domestic_opening_gross']*['Is_Sequel'],
['domestic_opening_gross']*['Twentieth Century Fox'],
['domestic_opening_gross']*['Walt Disney Studios Motion Pictures'],
['domestic_opening_gross']*['Warner Bros.'],
['domestic_opening_gross']*['Adventure'],
['domestic_opening_gross']*['Comedy'], 
['foreign_market_count']*['Walt Disney Studios Motion Pictures'],
['Years_Since_Release']*['Comedy'],
['Is_Adaptation']*['Walt Disney Studios Motion Pictures'],
['Is_Sequel']*['Action'],
['Is_Sequel']*['Adventure'],
['Is_Sequel']*['PG-13']
['Warner Bros.']*['Adventure'], 
['Action']*['PG-13'],
['Adventure']**2,
['Adventure']*['PG-13'], 
['Comedy']**2)

<>:5: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:19: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:5: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:19: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<ipython-input-9-ef797e530e07>:5: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  ['domestic_opening_gross']**2
<ipython-input-9-ef797e530e07>:19: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  ['Is_Sequel']*['PG-13']


TypeError: 'int' object is not subscriptable

In [69]:
lasso_df = linear_df[['Foreign_Gross',
                      'Title',
                      'domestic_opening_gross', 
                      'foreign_market_count',
                      'Adventure',
                      'Comedy',
                      'Years_Since_Release',
                      'Is_Sequel',
                      'Twentieth Century Fox',
                      'Walt Disney Studios Motion Pictures',
                      'Warner Bros.',
                      'Is_Adaptation',
                      'Action',
                      'PG-13']]

In [70]:
lasso_df

,Foreign_Gross,Title,domestic_opening_gross,foreign_market_count,Adventure,Comedy,Years_Since_Release,Is_Sequel,Twentieth Century Fox,Walt Disney Studios Motion Pictures,Warner Bros.,Is_Adaptation,Action,PG-13
0,391683624,Warcraft,24166110,66,0,0,4,0,0,0,0,0,1,1
1,187700000,Bridget Jones's Baby,8571785,61,0,1,4,0,0,0,0,0,0,0
2,301219864,xXx: Return of Xander Cage,20130142,55,0,0,3,1,0,0,0,0,1,1
3,289393001,The Great Wall,18469620,64,0,0,4,0,0,0,0,0,1,1
4,198923741,Mr. Bean's Holiday,9889780,68,0,1,13,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,33347096,The Longest Yard,47606480,52,0,1,15,0,0,0,0,0,0,1
843,53248834,The Blind Side,34119372,33,0,0,11,0,0,0,1,0,0,1
844,19530702,Ride Along,41516170,19,0,0,6,0,0,0,0,0,1,1
845,179067330,The Impossible,143818,47,0,0,8,0,0,0,0,0,0,1


In [71]:
lasso_df.dtypes

Foreign_Gross                           int64
Title                                  object
domestic_opening_gross                  int64
foreign_market_count                    int64
Adventure                               int64
Comedy                                  int64
Years_Since_Release                     int64
Is_Sequel                               int64
Twentieth Century Fox                   int64
Walt Disney Studios Motion Pictures     int64
Warner Bros.                            int64
Is_Adaptation                           int64
Action                                  int64
PG-13                                   uint8
dtype: object

In [72]:
lasso_df['domestic_opening_gross_X2'] = lasso_df['domestic_opening_gross']**2

lasso_df['dom_X_foreign_market'] = lasso_df['domestic_opening_gross']*lasso_df['foreign_market_count']

lasso_df['dom_X_Sequel']=lasso_df['domestic_opening_gross']*lasso_df['Is_Sequel']

lasso_df['dom_X_Fox'] = lasso_df['domestic_opening_gross']*lasso_df['Twentieth Century Fox']

lasso_df['dom_X_Disney']=lasso_df['domestic_opening_gross']*lasso_df['Walt Disney Studios Motion Pictures']

lasso_df['dom_X_Warner']=lasso_df['domestic_opening_gross']*lasso_df['Warner Bros.']

lasso_df['dom_X_Adventure']=lasso_df['domestic_opening_gross']*lasso_df['Adventure']

lasso_df['dom_X_Comedy']=lasso_df['domestic_opening_gross']*lasso_df['Comedy']

lasso_df['foreign_X_Disney']=lasso_df['foreign_market_count']*lasso_df['Walt Disney Studios Motion Pictures']

lasso_df['Years_X_Comedy']=lasso_df['Years_Since_Release']*lasso_df['Comedy']

lasso_df['Adaptation_X_Disney']=lasso_df['Is_Adaptation']*lasso_df['Walt Disney Studios Motion Pictures']

lasso_df['Sequel_X_Action']=lasso_df['Is_Sequel']*lasso_df['Action']

lasso_df['Sequel_X_Adventure']=lasso_df['Is_Sequel']*lasso_df['Adventure']

lasso_df['Sequel_X_PG-13']=lasso_df['Is_Sequel']*lasso_df['PG-13']

lasso_df['Warner_X_Adventure']=lasso_df['Warner Bros.']*lasso_df['Adventure'] 

lasso_df['Action_X_PG-13']=lasso_df['Action']*lasso_df['PG-13']

lasso_df['Adventure_X2']=lasso_df['Adventure']**2

lasso_df['Adventure_X_PG-13']=lasso_df['Adventure']*lasso_df['PG-13'] 

lasso_df['Comedy_X2']=lasso_df['Comedy']**2


<ipython-input-72-d9539e1157ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lasso_df['domestic_opening_gross_X2'] = lasso_df['domestic_opening_gross']**2
<ipython-input-72-d9539e1157ba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lasso_df['dom_X_foreign_market'] = lasso_df['domestic_opening_gross']*lasso_df['foreign_market_count']
<ipython-input-72-d9539e1157ba>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [73]:
lasso_df.drop([ 'foreign_market_count',
                      'Is_Sequel',
                      'Twentieth Century Fox',
                      'Walt Disney Studios Motion Pictures',
                      'Warner Bros.',
                      'Is_Adaptation',
                      'Action',
                      'PG-13'],axis=1, inplace = True)

In [74]:
lasso_df.columns

Index(['Foreign_Gross', 'Title', 'domestic_opening_gross', 'Adventure',
       'Comedy', 'Years_Since_Release', 'domestic_opening_gross_X2',
       'dom_X_foreign_market', 'dom_X_Sequel', 'dom_X_Fox', 'dom_X_Disney',
       'dom_X_Warner', 'dom_X_Adventure', 'dom_X_Comedy', 'foreign_X_Disney',
       'Years_X_Comedy', 'Adaptation_X_Disney', 'Sequel_X_Action',
       'Sequel_X_Adventure', 'Sequel_X_PG-13', 'Warner_X_Adventure',
       'Action_X_PG-13', 'Adventure_X2', 'Adventure_X_PG-13', 'Comedy_X2'],
      dtype='object')

In [75]:
len(list(lasso_df.columns))

25

In [77]:
X, y = lasso_df.drop(['Foreign_Gross', 'Title'], axis=1), lasso_df['Foreign_Gross']
#split all data into X and y (for train+validate) and X and y test
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=7)
#split the train+validate--X and y again into separate train and validate buckets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3)

In [78]:
#set up linear regression
lm = LinearRegression()

#validate

lm.fit(X_train, y_train)
print(f'Linear Regression TRAIN R^2: {lm.score(X_train, y_train):.3f}')
print(f'Linear Regression VAL R^2: {lm.score(X_val, y_val):.3f}')

Linear Regression TRAIN R^2: 0.639
Linear Regression VAL R^2: 0.560


#### This looks good, but not as good as our earlier models, so let's see if we can use intuition to narrow down on these features, since there is overfitting indicated by the Train/Val ratio

In [140]:
#removing features one by one to see which impact our regression below.

X, y = lasso_df.drop(['Foreign_Gross', 'Title',
                     'Adventure',
                      'Comedy', 
                      #'Years_Since_Release', 
                      'domestic_opening_gross_X2',
                      #'dom_X_foreign_market', 
                      #'dom_X_Sequel', 
                      'dom_X_Fox', 
                      #'dom_X_Disney',
                      'dom_X_Warner', 
                      #'dom_X_Adventure', 
                      #'dom_X_Comedy', #KEEP
                      #'foreign_X_Disney',
                      #'Years_X_Comedy', 
                      #'Adaptation_X_Disney', 
                      #'Sequel_X_Action', #KEEP
                      #'Sequel_X_Adventure', #KEEP
                      'Sequel_X_PG-13', 
                      'Warner_X_Adventure',
                      #'Action_X_PG-13', 
                      'Adventure_X2', 
                      'Adventure_X_PG-13', 
                      'Comedy_X2'
                     
                     ], axis=1), lasso_df['Foreign_Gross']
#split all data into X and y (for train+validate) and X and y test
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=7)
#split the train+validate--X and y again into separate train and validate buckets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3)

In [141]:
#set up linear regression
lm = LinearRegression()

#validate

lm.fit(X_train, y_train)
print(f'Linear Regression TRAIN R^2: {lm.score(X_train, y_train):.3f}')
print(f'Linear Regression VAL R^2: {lm.score(X_val, y_val):.3f}')

Linear Regression TRAIN R^2: 0.632
Linear Regression VAL R^2: 0.593


#### The numbers are opimized as much as possible manually, but does make a difference!
#### Let's cross validate as well

In [145]:
def cv_results(X, y, model=LinearRegression(normalize=True), scoring='r2', rounding=4, cv=5, minimize_score=False):
    '''
    For a set of features and target X, y, perform a 5 fold cross validation.
    Fit and validate a model, and report results
    Note: `sklearn.model_selection.cross_validate` defaults to 5 fold, 80/20 validation splits.
    '''
    
    model_cv = cross_validate(model, X, y, cv=cv, return_train_score=True, 
                              scoring=scoring, return_estimator=True)

    if minimize_score:
        select = model_cv['test_score'].argmin()
    else:
        select = model_cv['test_score'].argmax()

    final_model = model_cv['estimator'][select]
    
    # Capture the ratio between the train and test scores to understand possible under/over fitting
    model_cv['train_test_score_ratio'] = model_cv['train_score'] / model_cv['test_score']
    
    mean_score = round(model_cv['test_score'].mean(), rounding)
    min_score = round(model_cv['test_score'].min(), rounding)
    max_score = round(model_cv['test_score'].max(), rounding)
    
    mean_ratio = round(model_cv['train_test_score_ratio'].mean(), rounding)
    min_ratio = round(model_cv['train_test_score_ratio'].min(), rounding)
    max_ratio = round(model_cv['train_test_score_ratio'].max(), rounding)
    
    # report results
    print(f'Test {scoring} {":":<15} Mean = {mean_score}\tRange = ({min_score}, {max_score})')
    print(f'Train/Test {scoring} Ratio {":":<3} Mean = {mean_ratio}\tRange = ({min_ratio}, {max_ratio})')
    
    print('\nBest Model Feature coefficient results:')
    for feature, coef in zip(X.columns, final_model.coef_):
        print(f'{feature + ":":<16} {coef:.2f}')

In [146]:
cv_results(X,y)

Test r2 :               Mean = 0.5873	Range = (0.3781, 0.7326)
Train/Test r2 Ratio :   Mean = 1.1483	Range = (0.8059, 1.7255)

Best Model Feature coefficient results:
domestic_opening_gross: 1.38
Years_Since_Release: -2342043.13
dom_X_foreign_market: 0.03
dom_X_Sequel:    0.06
dom_X_Disney:    0.54
dom_X_Adventure: 1.07
dom_X_Comedy:    -2.36
foreign_X_Disney: -201927.13
Years_X_Comedy:  2253496.24
Adaptation_X_Disney: 65330764.27
Sequel_X_Action: 28147796.03
Sequel_X_Adventure: 46536538.28
Action_X_PG-13:  32417643.00


### Our highest R^2 yet! 
#### Our Degree 2 Polynomian score is even higher than our manually crafted interaction features model from the "Regression Model- Interactions" notebook:

Linear Regression TRAIN R^2: 0.617

Linear Regression VAL R^2: 0.585

#### This is our final model, before we go back to our Regression: Main notebook to take a look at residuals

In [148]:
#pickling our original df for reference
lasso_df.to_pickle('lasso_df.pickle')

In [151]:
#dropping the variables we didn't need before we move onto residuals in our other notebook

residual_df = lasso_df.drop(['Adventure',
                      'Comedy', 
                      'domestic_opening_gross_X2',
                      'dom_X_Fox', 
                      'dom_X_Warner', 
                      'Sequel_X_PG-13', 
                      'Warner_X_Adventure',
                      'Adventure_X2', 
                      'Adventure_X_PG-13', 
                      'Comedy_X2'], axis = 1)


In [152]:
#pickling our newest df for residuals
residual_df.to_pickle('residual_df.pickle')